색상 히스토그램을 기반으로 주어진 이미지들 중 비슷한 색상 분포를 가지고 있는 이미지를 찾아주는 기능을 구현하는게 목표입니다! 

## 학습목표 

+ 컴퓨터에서 이미지가 표현되는 방식을 이해한다. 
+ pillow와 opencv로 python에서 이미지 파일을 열고 정보를 추출할 수 있다. 
+ CIFAR-100에서 히스토그램을 기반으로 유사한 이미지를 골라낼 수 있다. 



### 용어정리

**화소(pixel)**: 디지털 화면에 표시된 수많은 점들 중 색상을 가지는 하나의 점을 나타냅니다.       
**레스터(raster) & 비트맵(bitmap)**: 픽셀을 RGB값으로 하나하나 색상을 저장하는 방식

**RGB**: 각 화소는 세 개의 단일 색의 강도를 각각 조절하여 색상을 표현합니다. 
**컬러스페이스**: 색을 표현하는 다양한 방식 
**채널**:  RGB 
**tqdm**: 진행상황 표시바  


[opencv공식문서 ]https://docs.opencv.org/4.3.0/d8/dfe/classcv_1_1VideoCapture.html#aabce0d83aa0da9af802455e8cf5fd181


### pickle 

* 텍스트 상태가 아닌 데이터를 저장할 때 사용합니다. 
* 장점: 객체자체를 바이너리 형식으로 저장이 가능합니다. 

### OpenCV

* cv용 라이브러리입니다. 영상처리에 유용한 고급 기능이 있습니다. 
* 다양한 언어로 호출이 가능합니다. 

### 20-2. 디지털 이미지 

![image](https://user-images.githubusercontent.com/68840414/108644634-274a7c80-74f3-11eb-8fa2-68512c0b59c4.png)

인간의 눈에 있는 시세포가 대부분 세 가지로 이루어져 있기 때문에 RBG로 나타냅니다. 

![image](https://user-images.githubusercontent.com/68840414/108644662-4a752c00-74f3-11eb-9ae1-dc07b21e35fb.png)


### 이미지를 저장하는 방식 2가지 

1) 단순하게 RGB로 각각의 점들을 표현한다. 

 RGB로 표현된 점들로 이미지를 저장하는 가장 단순한 방법은 ** 각 점 하나하나의 색상 값을 저장하는 방식**입니다.     
 보통 한 점마다 각 색사아별로 8비트를 사용하여 0~255 사이의 값으로 해당 색의 감도를 표시합니다. 

2) 벡터로 표현할 수 있습니다. 

이미지를 상대적인 점과 선의 위치를 방정식으로 기록해두었다가, 확대, 축소에 따라 디지털 화면의 각 화소에 어떻게 표현될지를 재계산하는 방법으로    
이미지가 깨지지 않습니다. 


일반적으로는 래스터 방식으로 저장하며, 확대, 축소해도 안 깨지는 것은 벡터방식으로 이미지를 저장해 놓은 것입니다. 
![image](https://user-images.githubusercontent.com/68840414/108645653-f0766580-74f6-11eb-958a-bbbebd7e1a19.png)


인간의 눈은 색상보다는 음영으로 사물을 구분합니다. 

#### 래스터 이미지 저장방식 

* YUV: 흑백에 1/4의 해상도를 가진 두 색상 채널을 덧붙여서 송출하는 방식      
* HSV: 색감을 수치적으로 조작할 때 더 직관적으로 이해 가능한 방식(색상, 채도, 명도)
* CMYK: 인쇄매체에서 자주 사용하는 검정색을 컬러로 섞어서 표현하면 잉크낭비가 심하다는 이유로 4가지 색상을 사용하여 나타내는 방식


컬러스페이스에 RGB  채널로 색을 표현합니다. 그런데 이렇게 표현하는 방법은 용량을 많이 차지합니다. 

#### 압축파일별 이미지 저장방식

* JPEG: 근처에 있는 화소들을 묶어, 비슷한 색들을 뭉뚱그리는 방식으로 이미지 압축 
    단점: 색상정보의 손실, 저장시 압축률을 높여거나 압축에 압축을 해야함. 
    
* PNG: 색상 손실없이 이미지 압축,  팔레트 형태로 색을 구분하여 단순하면 용량을 적게 사용하지만 사진처럼 색상이 많으면 용량도 증가

* GIF: 이미지 내에 여러 프레임을 사용, 색상정보 또한 손실없이 사용, 256개의 색상만 사용가능 

## 20-3. Pillow 사용법 

### Pillow의 특징 

1) numpy와 결합하여 간편하게 사용할 수 있습니다. 

이미지는 배열 형태의 데이터입니다. 데이터 타입은 unit8 8비트 정수가 되어 0 ~ 255사이의 값으로 나타냅니다. 255로 갈수록 하얀색이 됩니다. 



In [19]:
import numpy as np
from PIL import Image


#np.zeros() 모든 채널의 값을 0으로(검정색)으로 나옵니다. 

data = np.zeros([32,32,3], dtype=np.uint8)#배열형태로 만들기 
image = Image.fromarray(data,'RGB') # Image.fromarray()를 통해 이미지 객체로 변환합니다. 
image.show()
print(len(data))

32


In [20]:
# 빨간색 바탕으로 만들어보도록 하겠습니다. 

#np.zeros()는 32개의 점이 있고, 그것들이 각각 어떤 RGB색이 얼마나 혼합되어있는지를 나타냅니다. 
#data[:,:]로 나타낸다면 배열을 전체 슬라이싱 하겠다는 의미입니다. 그리고 첫번째는 모두 255로 지정하겠다! 

data[:,:] = [255,0,0]
image = Image.fromarray(data,'RGB')
image.show()
data

array([[[255,   0,   0],
        [255,   0,   0],
        [255,   0,   0],
        ...,
        [255,   0,   0],
        [255,   0,   0],
        [255,   0,   0]],

       [[255,   0,   0],
        [255,   0,   0],
        [255,   0,   0],
        ...,
        [255,   0,   0],
        [255,   0,   0],
        [255,   0,   0]],

       [[255,   0,   0],
        [255,   0,   0],
        [255,   0,   0],
        ...,
        [255,   0,   0],
        [255,   0,   0],
        [255,   0,   0]],

       ...,

       [[255,   0,   0],
        [255,   0,   0],
        [255,   0,   0],
        ...,
        [255,   0,   0],
        [255,   0,   0],
        [255,   0,   0]],

       [[255,   0,   0],
        [255,   0,   0],
        [255,   0,   0],
        ...,
        [255,   0,   0],
        [255,   0,   0],
        [255,   0,   0]],

       [[255,   0,   0],
        [255,   0,   0],
        [255,   0,   0],
        ...,
        [255,   0,   0],
        [255,   0,   0],
        [255,   0,   0]]

In [16]:
# 128짜리 흰색이미지 만들기 

data1 = np.zeros([128,128,3],dtype=np.uint8)
image1 = Image.fromarray(data1, 'RGB')
image1.show()

data1[:,:] = [255,255,255] #흰색이니깐 
image1 = Image.fromarray(data1,'RGB')
image1.show()

In [28]:
# 실습하기 

from PIL import Image
import os

#연습용 파일 경로 
image_path = os.getenv('HOME') + '/aiffel/python_image_proc/pillow_practice.png'

#이미지 열기 
image = Image.open(image_path)
image

# width와 height출력 

print(image.width) #이미지의 가로 
print(image.height)#이미지의 세로 

#jpg파일 형식으로 저장하기 

image_path1 = os.getenv('HOME') + '/aiffel/python_image_proc/pillow_practice.jpg'
image1 = image.convert('RGB') # 이미지를 변하게 한다. 
image1.save(image_path1)

620
465


In [34]:
# resize를 이용해서이미지의 크기를 100*200으로 조정
resize_image1 = image1.resize((100,200)) #jpg로 저장된 이미지 사이즈를 조정합니다. 
resize_image1.show()

#저장하기 
resize_image_path = os.getenv("HOME") + '/aiffel/python_image_proc/pillow_practice.png' #이미지 저장경로에서 이미지를 찾아와서 
resize_image1.save(resize_image_path) #그 곳에 저장합니다. 

In [38]:
#눈부분만 crop하기 
#눈 위치은 (300,100,600,400)


box = (300,100,600,400) #crop할 부분의 박스의 위치를 저장합니다. 
crop_image = image1.crop(box)
crop_image.show()

crop_image_path = os.getenv('HOME') + '/aiffel/python_image_proc/pillow_practice_cropped.png' #crop한 파일을 저장합니다. 
crop_image.save(crop_image_path)


### 20-4. Pillow를 활용한 데이터 전처리 

CIFAR-100 데이터를 받아 개별 이미지 파일로 추출하기 

이미지 데이터베이스 구축을 위해 **CIFAR-100데이터 셋**을 활용하겠습니다. 

### CIFAR-100데이터셋 

* 32*32 이미지들이 100개의 클래스당 600개(학습용500, test용 100) 6만장으로 구성된 데이터 셋입니다. 
* 데이터 셋을 풀면 meta, train,test이렇게 3가지로 분류되어 나옵니다. 

In [41]:
# cifar-100데이터셋에서 

import os 
import pickle

dir_path = os.getenv('HOME')+'/aiffel/python_image_proc/cifar-100-python'
train_file_path = os.path.join(dir_path,'train')

with open(train_file_path,'rb')as f:
    train = pickle.load(f, encoding='bytes')
    
print(train)
print(type(train))

{b'filenames': [b'bos_taurus_s_000507.png', b'stegosaurus_s_000125.png', b'mcintosh_s_000643.png', b'altar_boy_s_001435.png', b'cichlid_s_000031.png', b'phone_s_002161.png', b'car_train_s_000043.png', b'beaker_s_000604.png', b'fog_s_000397.png', b'rogue_elephant_s_000421.png', b'computer_keyboard_s_000757.png', b'willow_tree_s_000645.png', b'sunflower_s_000549.png', b'palace_s_000759.png', b'adriatic_s_001782.png', b'computer_keyboard_s_001277.png', b'bike_s_000682.png', b'wolf_pup_s_001323.png', b'squirrel_s_002467.png', b'sea_s_000678.png', b'shrew_s_002233.png', b'pine_tree_s_000087.png', b'rose_s_000373.png', b'surveillance_system_s_000769.png', b'pine_s_001533.png', b'table_s_000897.png', b'opossum_s_001237.png', b'quercus_alba_s_000257.png', b'leopard_s_000414.png', b'possum_s_002195.png', b'bike_s_000127.png', b'balmoral_castle_s_000361.png', b'acer_saccharinum_s_000646.png', b'lapin_s_000916.png', b'chimp_s_001419.png', b'clock_s_002291.png', b'streetcar_s_000663.png', b'male_c

In [42]:
train.keys()

dict_keys([b'filenames', b'batch_label', b'fine_labels', b'coarse_labels', b'data'])

In [43]:
#각 키들이 문자열이 아닌 `b`로 시작하는 `bytes`로 됩니다. 

type(train[b'filenames']) #예상한대로 리스트라는데 나는 왜 예상이 안 된걸까요..ㅎㅎㅎ

list

In [45]:
train[b'filenames'][0:5]

#파일이름이 나오네요. 그런데 다 b가 붙어있네요.

[b'bos_taurus_s_000507.png',
 b'stegosaurus_s_000125.png',
 b'mcintosh_s_000643.png',
 b'altar_boy_s_001435.png',
 b'cichlid_s_000031.png']

In [48]:
image2 = Image.open(train[b'filenames'][0])
image2
#이러면 나올 줄 알았는데 아니네요.

FileNotFoundError: [Errno 2] No such file or directory: b'bos_taurus_s_000507.png'

In [51]:
#배열형태로 출력이 됩니다. 
len(train[b'data'][0]) #길이가 나타내는 의미는 무엇일까요? 3072

train[b'data'][0].shape #여기서 3072라는 숫자는 32*32*3으로 R이 1024 G가 1024, B가 1024로 나와 있고 순서대로 나열됩니다. 

(3072,)

In [62]:
#앞선 차원에서부터 데이터를 채우는 방식의 reshape을 위해 np.reshape에서 order라는 인자를 사용합니다. 

image_data = train[b'data'][4].reshape([32,32,3], order='F')
image = Image.fromarray(image_data)#배열을 이미지 객체로 변환시켜 줍니다. 
image.show()

In [59]:
#눈썰미가 좋은 사람만 보인다는데 x축과 y축이 뒤집어져 있다네요. 0번째 이미지는 잘 모르겠어서 4번째이미지를 불러와 보았습니다. 
#그러네요. x,y축이 뒤집어져있네요. 
#축을 바꾸는 건 np.swapaxes(0,1)이 유용하다고 하니 사용해보도록 하겠습니다. 

In [69]:
image_data = image_data.swapaxes(0,1)
image = Image.fromarray(image_data)
image.show() #변경이 잘 되네요. 축을 바꿀때는 np.swapaxes(0,1)

In [85]:
# 차례차례 numpy 배열로 읽어서 이미지 파일로 저장해주자! 

from tqdm import tqdm
import os
from PIL import Image
import pickle
import numpy as np

dir_path = os.getenv("HOME") + '/aiffel/python_image_proc/cifar-100-python'
train_file_path = os.path.join(dir_path,'train')

#image를 저장할 cifar-100-python의 하위 디렉토리를 생성해봅시다!

images_dir_path = os.path.join(dir_path,'images')
if not os.path.exists(images_dir_path):
    os.mkdir(images_dir_path)

# 32*32의 이미지파일 5만개 생성하기

with open(train_file_path,'rb') as f:#바이너리 파일을 읽으려면 rb로 쓰기는 wb
    train = pickle.load(f, encoding='bytes')
    for i in tqdm(range(len(train[b'filenames']))):
        
        filename = train[b'filenames'][i].decode()
        data = train[b'data'][i].reshape([32, 32, 3], order='F')
        image = Image.fromarray(data.swapaxes(0,1))
        image.save(os.path.join(images_dir_path,filename))

        
# with open(train_file_path, 'rb') as f:
#     train = pickle.load(f, encoding='bytes')
#     for i in tqdm(range(len(train[b'filenames']))):
        
#         filename = train[b'filenames'][i].decode()
#         data = train[b'data'][i].reshape([32, 32, 3], order='F')
#         image = Image.fromarray(data.swapaxes(0, 1))
#         image.save(os.path.join(images_dir_path, filename))        


100%|██████████| 50000/50000 [00:13<00:00, 3765.82it/s]


### 20_5. OpenCV (1) 안녕, OpenCV

* opencv는 컴퓨터 비전용 고급 기능이 있는 라이브러리입니다. 
* 이미지는 `[높이,넓이,채널]`로 구성된 배열이기에 컴퓨터 비전에서는 이 배열 데이터를 어떻게 처리할지가 중요합니다.  
<br/>

* 이번 노드에서는 opencv에서 특정 색을 지닌 영역만 추출해보도록 하겠습니다. 
* 관심있는 부분이 특정 색을 지니고 있다면, 원하는 부분을 배경으로 구분하고 그 부분만 따로 떼어냅니다. 
* 카메라를 통해 영상을 읽고, 파란색을 찾기 쉽도록 컬러스페이스를 HSV로 변환한 뒤, 해당 색상과 맞는 영역만 표시하는 작업을 진행합니다. 


### 20_6. OpenCV (2)톺아보기 

In [ ]:
import cv2 as cv #현재 cv4.x대 이지만 불러올 때 cv2로 불러옵니다. 
import numpy as np #opencv와 잘 맞는 numpy를 호출합니다. 

#이미지 데이터는 숫자행렬입니다. 

cap = cv.VideoCapture(0)

while(1):

    # Take each frame
    _, frame = cap.read()

    # Convert BGR to HSV
    hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)

    # define range of blue color in HSV
    lower_blue = np.array([110,50,50])
    upper_blue = np.array([130,255,255])

    # Threshold the HSV image to get only blue colors
    mask = cv.inRange(hsv, lower_blue, upper_blue)

    # Bitwise-AND mask and original image
    res = cv.bitwise_and(frame,frame, mask= mask)
    cv.imshow('frame',frame)
    cv.imshow('mask',mask)
    cv.imshow('res',res)
    k = cv.waitKey(5) & 0xFF
    if k == 27:
        break

cv.destroyAllWindows()
cap.release()

# 회고

다음에 이 노드를 진행하게 된다면... 

1) opencv 20-6부터 천천히 보기 

2) 실습을 내 힘으로 진행해보기 